# Tiling

### Before the data can be used in the model, they have to tiled to 256x256


https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7

TODO
1) Tif lesen, georeferenzierten Punkt oben links für späteres Nutzen erstellen 
2) Georeferenzierung fallen lassen
3) tiling
4) Klassifizierung 
5) wieder zusammensetzten der Tiles 
6) Georeferenzierung wieder hinzufügen 

In [1]:
import os
import sys
from PIL import Image
from osgeo import gdal
import glob
import rasterio
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import numpy as np

In [2]:
savedir = r"E:/W_Katrin/Kigali/Raster/tiles"
filename = r"E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"

## Idea 0
#### gdal translate

In [11]:
in_ds = gdal.Open(filename)
out_ds = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_2.jpg"
ds=gdal.Translate(out_ds, in_ds, 
                  format = "JPEG", 
                  bandList = [1,2,3], 
                  width = in_ds.RasterXSize, 
                  height = in_ds.RasterYSize, 
                  rgbExpand = "rgb")

In [6]:
in_ds = gdal.Open(filename)


In [10]:
in_ds.RasterYSize

8267

In [17]:
out_ds = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg"

In [36]:
!gdal_translate -of JPEG -b 1 -b 2 -b 3 -scale - a_srs None E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg

Usage: gdal_translate [--help-general] [--long-usage]
       [-ot {Byte/Int16/UInt16/UInt32/Int32/Float32/Float64/
             CInt16/CInt32/CFloat32/CFloat64}] [-strict]
       [-if format]* [-of format]
       [-b band] [-mask band] [-expand {gray|rgb|rgba}]
       [-outsize xsize[%]|0 ysize[%]|0] [-tr xres yres]
       [-r {nearest,bilinear,cubic,cubicspline,lanczos,average,mode}]
       [-unscale] [-scale[_bn] [src_min src_max [dst_min dst_max]]]* [-exponent[_bn] exp_val]*
       [-srcwin xoff yoff xsize ysize] [-epo] [-eco]
       [-projwin ulx uly lrx lry] [-projwin_srs srs_def]
       [-a_srs srs_def] [-a_coord_epoch epoch]
       [-a_ullr ulx uly lrx lry] [-a_nodata value]
       [-a_scale value] [-a_offset value]
       [-nogcp] [-gcp pixel line easting northing [elevation]]*
       |-colorinterp{_bn} {red|green|blue|alpha|gray|undefined}]
       |-colorinterp {red|green|blue|alpha|gray|undefined},...]
       [-mo "META-TAG=VALUE"]* [-q] [-sds]
       [-co "NAME=VALUE"]* [-st

ERROR 6: Too many command options 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg'


In [32]:
with rasterio.open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg") as src: 
    meta = src.meta
    print(meta)

{'driver': 'JPEG', 'dtype': 'uint8', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 3, 'crs': CRS.from_epsg(32736), 'transform': Affine(0.4963054573117089, 0.0, 173723.36172185623,
       0.0, -0.4963054573117089, 9789230.701895246)}


In [34]:
dst = gdal.Open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg")
dst.GetMetadata()
!gdalinfo E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg

Driver: JPEG/JPEG JFIF
Files: E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg
       E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg.aux.xml
Size is 13162, 8267
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 36S",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 36S",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",33,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG

## Idea 1

In [5]:
raster = gdal.Open(filename)

In [6]:
band1 = raster.GetRasterBand(1)
rows = raster.RasterYSize
cols = raster.RasterXSize

b1_data = band1.ReadAsArray(0,0,cols, rows)

In [7]:
#print(b1_data)

In [8]:
band2 = raster.GetRasterBand(2)
b2_data = band2.ReadAsArray(0,0,cols, rows)

In [9]:
band3 = raster.GetRasterBand(3)
b3_data = band3.ReadAsArray(0,0,cols, rows)

In [10]:
rgb_array = np.array([b1_data, b2_data, b3_data])

In [17]:
rgb_array


array([[[547.3768 , 557.5864 , 558.6454 , ..., 601.21313, 606.2491 ,
         578.84186],
        [541.05414, 544.25433, 542.46704, ..., 603.34283, 592.30634,
         560.804  ],
        [533.61566, 530.57184, 530.2392 , ..., 584.64484, 568.3835 ,
         542.2961 ],
        ...,
        [679.7468 , 657.0126 , 652.2072 , ..., 848.4657 , 844.2835 ,
         845.85815],
        [681.76044, 664.20935, 662.334  , ..., 862.1436 , 853.9357 ,
         847.71747],
        [679.9162 , 663.1915 , 662.75006, ..., 877.46564, 862.4148 ,
         852.59845]],

       [[575.74634, 582.7441 , 582.06665, ..., 612.41003, 613.8362 ,
         581.7774 ],
        [579.1492 , 580.11993, 577.3272 , ..., 616.54407, 603.1896 ,
         569.0417 ],
        [581.32385, 576.9154 , 575.74567, ..., 603.93286, 586.14636,
         556.4349 ],
        ...,
        [571.11475, 554.5059 , 558.92236, ..., 706.9965 , 705.4458 ,
         709.72473],
        [565.23   , 556.0568 , 565.5616 , ..., 711.4415 , 707.4643 ,
   

In [18]:
# reshape the array to [[[b1, b2, b3]x13162]x8267]


(8267, 13162, 3)

In [12]:
print(b2_data)
print(rgb_array[1])

[[575.74634 582.7441  582.06665 ... 612.41003 613.8362  581.7774 ]
 [579.1492  580.11993 577.3272  ... 616.54407 603.1896  569.0417 ]
 [581.32385 576.9154  575.74567 ... 603.93286 586.14636 556.4349 ]
 ...
 [571.11475 554.5059  558.92236 ... 706.9965  705.4458  709.72473]
 [565.23    556.0568  565.5616  ... 711.4415  707.4643  706.12933]
 [560.28436 552.8299  565.97186 ... 717.3969  709.27527 706.0574 ]]
[[575.74634 582.7441  582.06665 ... 612.41003 613.8362  581.7774 ]
 [579.1492  580.11993 577.3272  ... 616.54407 603.1896  569.0417 ]
 [581.32385 576.9154  575.74567 ... 603.93286 586.14636 556.4349 ]
 ...
 [571.11475 554.5059  558.92236 ... 706.9965  705.4458  709.72473]
 [565.23    556.0568  565.5616  ... 711.4415  707.4643  706.12933]
 [560.28436 552.8299  565.97186 ... 717.3969  709.27527 706.0574 ]]


In [14]:
# changing to Int

random_array = (rgb_array *255).astype(np.uint8)


In [15]:
# try: save as PIL Image [white image] (maybe here same problem like matplotib
from PIL import Image
im = Image.fromarray(random_array).convert('RGB')
print(im)
im.show()

#im.save("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_rgb.jpg")

TypeError: Cannot handle this data type: (1, 1, 13162), |u1

In [ ]:
# try: save with matplotlib
# for this solution I would need to get the array in the shape (12162, 8267, 3)
import matplotlib
matplotlib.pyplot.imsave("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_rgb2.jpg", rgb_array)

In [138]:
# try: writing as GeoTiff with GDAL
driver = gdal.GetDriverByName("GTiff")
output_path = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_multi_rgb2.tif"
outDs = driver.Create(output_path, cols, rows, bands=3)

In [173]:
outDs.GetRasterBand(1).WriteArray(b1_data)
outDs.GetRasterBand(2).WriteArray(b2_data)
outDs.GetRasterBand(3).WriteArray(b3_data)

"""
for i in range(3): 
    outband = outDs.GetRasterBand(i+1)
    outband.WriteArray(rgb_array[i])
"""

AttributeError: 'NoneType' object has no attribute 'GetRasterBand'

In [141]:
outDs = None


## Idea 2: Try with rasterio and gdal

In [12]:
raster = rasterio.open(filename)
out_meta = raster.meta.copy()
print(out_meta)
out_meta.update({"driver": "GTiff",
                 "count":1})

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 5, 'crs': CRS.from_epsg(32736), 'transform': Affine(0.4963054573117089, 0.0, 173723.36172185623,
       0.0, -0.4963054573117089, 9789230.701895246)}


In [37]:
#write raster 1
b1 = raster.read(1)
with rasterio.open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B1.tiff", "w", driver="GTiff", height=raster.shape[0], width = raster.shape[1], count = 1, dtype = "float32", nodata = "NaN") as dst: 
    dst.write(b1, 1)
b2=raster.read(2)
with rasterio.open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B2.tiff", "w", driver="GTiff", height=raster.shape[0], width = raster.shape[1], count = 1, dtype = float32,  nodata = "NaN") as dst: 
    dst.write(b2, 1)
b3 = raster.read(3)
with rasterio.open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B3.tiff", "w", driver="GTiff", height=raster.shape[0], width = raster.shape[1], count = 1, dtype = "float32", nodata = "NaN") as dst: 
    dst.write(b3, 1)


In [70]:
multi_bands = glob.glob("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B*[1:2:3]*.tiff")
multi_bands = ['E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B1.tiff',
 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B2.tiff',
 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B3.tiff']
multi_bands

['E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B1.tiff',
 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B2.tiff',
 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_B3.tiff']

In [71]:
multi_path = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_multi_rgb1.tiff"

In [ ]:
# try: stacking with eartyphy (not working)
rgb_stack = es.stack(multi_bands, multi_path)

In [73]:
#try: writing with rasterio [not working]
import rasterio

# Read metadata of first file
with rasterio.open(multi_bands[0]) as src0:
    meta = src0.meta
print(meta)
# Update meta to reflect the number of layers
meta.update({"driver": "GTiff", 
             "count":len(multi_bands)
            })

# Read each layer and write it to stack
with rasterio.open(multi_path, 'w', **meta) as dst:
    for id, layer in enumerate(multi_bands, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 1, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}


### some random lines

In [ ]:
img = Image.open(filename)
#tiling(filename, savedir, (256, 256))

In [18]:
from osgeo import gdal
dataset = gdal.Open(filename, gdal.GA_ReadOnly)
dataset

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F3BC19770> >

In [31]:
import cv2, os
base_path = "E:/W_Katrin/Kigali/Raster/"
new_path = "E:/W_Katrin/Kigali/Raster/"

#print(img)

im = gdal.Open(filename)
print(im)


outfile = filename.split('.')[0] + '.jpg'
print(outfile)
save(im, outputfile, "JPEG", quality=100)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000020F3F0FA160> >
E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.jpg


NameError: name 'save' is not defined

In [ ]:
from osgeo import gdal
import numpy
import rasterio

savedir = r"E:/W_Katrin/Kigali/Raster/tiles"
filename = r"E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"
dst_filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_rgb6.tiff"



src_ds = gdal.Open(filename)
dataset_src = src_ds.ReadRaster()

width = src_ds.RasterXSize
height=src_ds.RasterYSize
dataset = src_ds.ReadAsArray(0,0,width, height)
#print(dataset)
datasetrgb = dataset[:3]
#print(datasetrgb)

for band in range( src_ds.RasterCount ):
    band += 1
    print( "[ GETTING BAND ]: ", band)
    srcband = src_ds.GetRasterBand(band)
    if band == 3: 
        break
print(srcband)
"""
with rasterio.open(filename) as src: 
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "transform": None,
                 "count":3,
                 "crs": None}
                         )
print(out_meta)
with rasterio.open(dst_filename, "w", **out_meta) as dest1:
    dest1.write(datasetrgb)


driver = gdal.GetDriverByName("GTiff")

dst_ds = driver.Create(dst_filename, xsize=width, ysize=height, bands=3, options=["INTERLEAVE=PIXEL"])
with rasterio.open(dst_filename, "w", **out_meta) as dest1:
    dest1.write(srcband)
    
#dst_ds.WriteRaster(0,0,width,height,srcband,width,height,band_list=[1,2,3])

"""

In [ ]:
import rasterio
from rasterio.plot import show
import earthpy.spatial as es
import earthpy.plot as ep



savedir = r"E:/W_Katrin/Kigali/Raster/tiles"
filename = r"E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"
dst_filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_rgb8.tiff"


raster = rasterio.open(filename)
band1 = raster.read(1)
band2 = raster.read(2)
band3 = raster.read(3)

print(band1)
print(band2)
print(band3)

bands = [band1.tobytes(), band2.tobytes(), band3.tobytes()]

[[547.3768  557.5864  558.6454  ... 601.21313 606.2491  578.84186]
 [541.05414 544.25433 542.46704 ... 603.34283 592.30634 560.804  ]
 [533.61566 530.57184 530.2392  ... 584.64484 568.3835  542.2961 ]
 ...
 [679.7468  657.0126  652.2072  ... 848.4657  844.2835  845.85815]
 [681.76044 664.20935 662.334   ... 862.1436  853.9357  847.71747]
 [679.9162  663.1915  662.75006 ... 877.46564 862.4148  852.59845]]
[[575.74634 582.7441  582.06665 ... 612.41003 613.8362  581.7774 ]
 [579.1492  580.11993 577.3272  ... 616.54407 603.1896  569.0417 ]
 [581.32385 576.9154  575.74567 ... 603.93286 586.14636 556.4349 ]
 ...
 [571.11475 554.5059  558.92236 ... 706.9965  705.4458  709.72473]
 [565.23    556.0568  565.5616  ... 711.4415  707.4643  706.12933]
 [560.28436 552.8299  565.97186 ... 717.3969  709.27527 706.0574 ]]
[[616.11255 621.63556 619.8221  ... 618.7287  621.4599  593.7994 ]
 [617.0854  616.3494  612.34564 ... 632.1172  619.2708  588.62946]
 [617.74286 611.55493 609.7969  ... 624.94977 606.

In [16]:
from osgeo import gdal
import sys

src_ds = gdal.Open(filename)
if src_ds is None:
    print( 'Unable to open INPUT.tif')
    sys.exit(1)

print( "[ RASTER BAND COUNT ]: ", src_ds.RasterCount)
for band in range( src_ds.RasterCount ):
    band += 1
    print( "[ GETTING BAND ]: ", band)
    srcband = src_ds.GetRasterBand(band)
    if band == 3: 
        break


    stats = srcband.GetStatistics( True, True )
    if stats is None:
        continue

    print( "[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % ( \
                stats[0], stats[1], stats[2], stats[3] ))
print(srcband)

[ RASTER BAND COUNT ]:  5
[ GETTING BAND ]:  1
[ STATS ] =  Minimum=173.111, Maximum=4067.498, Mean=599.796, StdDev=185.006
[ GETTING BAND ]:  2
[ STATS ] =  Minimum=240.969, Maximum=3946.303, Mean=572.243, StdDev=123.608
[ GETTING BAND ]:  3
<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x000002601C9CEDF0> >


In [10]:
out_meta = raster.meta.copy()
print(out_meta)
# save out_meta for later use!!

out_meta.update({"driver": "JPEG",
                 "transform": None,
                 "crs": None}
                         )
print(out_meta)

outraster = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.jpg"
with rasterio.open(outraster, "w", **out_meta) as dest:
       dest.write(raster)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 5, 'crs': CRS.from_epsg(32736), 'transform': Affine(0.4963054573117089, 0.0, 173723.36172185623,
       0.0, -0.4963054573117089, 9789230.701895246)}
{'driver': 'JPEG', 'dtype': 'float32', 'nodata': nan, 'width': 13162, 'height': 8267, 'count': 5, 'crs': None, 'transform': None}


E:\W_Katrin\miniconda3\envs\tue_dl2\lib\site-packages\rasterio\__init__.py:230: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = writer(path, mode, driver=driver,


CPLE_NotSupportedError: JPEG driver doesn't support 5 bands.  Must be 1 (grey), 3 (RGB) or 4 bands (CMYK). 

Thoughts: 
JPG can only hold 3 layers --- 
should maybe drop two layers anyways as was also trained to be 3 layers


# TILING

In [12]:
# define tiling function 
def tiling(filename, savedir, target_size): 
    img = Image.open(filename)
    width, height = img.size
    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size
    frame_num = 1
    for col_i in range(0, width, w):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            save_to= os.path.join(savedir, "testing_{row}_{col}.jpg".format(row=row_i, col=col_i))
            crop.save(save_to)


### Andere Methode, hier fehlen Randpieces! (nur 270 tiles, bei oberem mehr tiles, auch Randtiles)

### This method results in a list of arrays

## TODO

develop code to enter many images and return tiles in a logical way (also think of mask pixels!!) 

Problem: we have images, but we need the mask to be tiled at the same location the exact same way!